In [1]:

# Descomprimir el completo 

UNZIP_DISABLE_ZIPBOMB_DETECTION=TRUE unzip archivo_comprimido_896.zip -d /home/yordonez/Danny_OV_WES/

# Para pasar de fastq.gz a fastq
gunzip *fastq.gz

### Si es necesario concatenar varios archivos de la misma muestra:
cat 102T_V350286221_L02_33_444771_1.fastq.gz 102T_V350286221_L03_33_444771_1.fastq.gz > 102T_444771_1.fastq.gz
cat 102T_V350286221_L02_33_444771_2.fastq.gz 102T_V350286221_L03_33_444771_2.fastq.gz > 102T_444771_2.fastq.gz


###concatenar todo
cat "CO-H,243121"*_L*_1.fastq > CO-E_243121_all_R1_fastq
cat "CO-E,275235"*_L*_1.fastq > CO-E_275235_all_R1_fastq

###Descargar en el pc:
scp -r yordonez3@biocompjord01.biosci.gatech.edu:/home/yordonez3/Danny_OV_WES/concatenated_fastq_faltantes/fastqc_results C:\\Users\\pmorales\\Downloads

###Descargar en el servidor:
scp "C:\Users\pmorales\Downloads\archivo_comprimido_1044.zip" yordonez3@bio-compjord01:/home/yordonez3/Danny_OV_WES/concatenated_fastq_faltantes/
scp "C:\Users\pmorales\Downloads\archivo_comprimido_1044.zip" yordonez3@130.207.66.26:/home/yordonez3/Danny_OV_WES/concatenated_fastq_faltantes/

Se procede a alinear con el genoma humano despues del control de calidad.



SyntaxError: invalid decimal literal (697790446.py, line 9)

In [ ]:
### 1. CONTROL DE CALIDAD1. ### FASTQC raw

conda activate fastqc_env
fastqc -o /home/yordonez3/Danny_OV_WES/concatenated_fastq/OVARIO *.fastq

#### Para correr muestras faltantes

# Activar el entorno de conda
conda activate fastqc_env

output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/fastqc_results"
input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/OVARIO"

# Iterar sobre los archivos .fastq.gz
for file in $input_dir/*_all_R2.fastq; do
    # Obtener el nombre del archivo sin la extensión
    filename=$(basename "$file" .fastq)

    # Verificar si los archivos de salida ya existen
    if [[ ! -f "$output_dir/$filename"_fastqc.html && ! -f "$output_dir/$filename"_fastqc.zip ]]; then
        # Si no existen los archivos de salida, ejecutar fastqc
        echo "Procesando $filename..."
        fastqc -o "$output_dir" "$file"
    else
        # Si los archivos de salida ya existen, saltar esta muestra
        echo "La muestra $filename ya ha sido procesada. Saltando..."
    fi
done


2. ### MULTIQC raw
multiqc total_samples_pool/EXOMA/FastQC_raw_ex -o total_samples_pool/EXOMA/MultiQC_raw_ex

2. ### MULTIQC raw
multiqc /home/yordonez3/Danny_OV_WES/concatenated_fastq_faltantes/fastqc_results/MultiQC_raw_ex

3. ###DescargarMultiQC en mi pc
scp -r yordonez3@bio-compjord01.biosci.gatech.edu:/home/yordonez3/Danny_OV_WES/concatenated_fastq_faltantes/ multiqc_data_1 /C:/Users/pmorales/Downloads/

In [2]:
### KRAKEN 2

# Directorio de entrada y salida
input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq_faltantes/"
output_dir="/home/yordonez3/Danny_OV_WES/kraken2_results"
kraken_db="/home/yordonez3/Danny_OV_WES/kraken2"

# Obtener los nombres de las muestras con dos patrones posibles (R1/R2 y _1/_2)
samples=($(ls $input_dir/*_R1.fastq | sed 's/_R1.fastq//'))

# Bucle para procesar cada muestra
for sample_path in "${samples[@]}"; do
    sample=$(basename "$sample_path")
    read1_file="${sample}_R1.fastq"
    read2_file="${sample}_R2.fastq"

    # Ejecutar Kraken 2
    kraken2 --db /home/yordonez3/Danny_OV_WES/kraken2 \
            --threads 10 \
            --output "${output_dir}/${sample}_kraken_output.txt" \
            --report "${output_dir}/${sample}_kraken_report.txt" \
            "$input_dir/$read1_file" "$input_dir/$read2_file"
    
    echo "Clasificación completada para la muestra $sample"
done

## Revisar resultados
multiqc kraken2 -o MQCkraken

### 3. ALINEAMIENTO BWA 

# 1. Indexar gemona de referencia 
#Toda la info del genoma : http://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/
        
# para descargarlo
wget http://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/hg38.fa.gz
gunzip hg38.fa.gz
du -h hg38.fa #para saber el peso (3.1GB)

# descargar el archivo de anotaciones
wget http://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/genes/hg38.refGene.gtf.gz
gunzip hg38.refGene.gtf.gz

#Para crear el index
# Entrar a la carpeta de salida y crear un enlace hacia donde esta el genoma
ln -s /home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/GRCh38_Gencode31/hg38.fa 

#Activar entorno conda
Conda activate

#Indexar: Indexar un genoma con bwa es preparar el archivo de referencia (por ejemplo, hg38.fa) para que el programa pueda buscar en él rápidamente las coincidencias con tus secuencias (fastq).
bwa index hg38.fa

#Se generan archivos:
#hg38.fa.amb
#hg38.fa.ann
#hg38.fa.bwt
#hg38.fa.pac
#hg38.fa.sa

## Correr el alineamiento

#Primero crear carpeta de salida si no existe
mkdir -p "OUTPUT_DIR"

# Rutas
FASTQ_DIR="/home/yordonez3/Danny_OV_WES/concatenated_fastq_faltantes"
REF_GENOME="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/GRCh38_Gencode31/hg38.fa"
OUTPUT_DIR="$FASTQ_DIR/BWA_data1"
REF_DIR="/home/yordonez3/Danny_OV_WES/GRCh38_Gencode31/index_BWA"


mkdir -p "$OUTPUT_DIR"

# Iterar sobre todos los archivos *_R1.fastq
for R1 in "$FASTQ_DIR"/*all_R1.fastq; do
    # Obtener nombre base (quita solo el _R1.fastq)
    base=$(basename "$R1" _R1.fastq)
    
    R2="$FASTQ_DIR/${base}all_R2.fastq"
    SAM="$OUTPUT_DIR/${base}.sam"
    BAM="$OUTPUT_DIR/${base}.bam"
    SORTED="$OUTPUT_DIR/${base}_sorted.bam"

    echo "🧬 Procesando muestra: $base"

    # Alineamiento
    bwa mem -t 8 "$REF_GENOME" "$R1" "$R2" > "$SAM"

    # Convertir SAM a BAM
    samtools view -bS "$SAM" > "$BAM"

    # Ordenar BAM
    samtools sort "$BAM" -o "$SORTED"
 
# Se ve algo como:
[bam_sort_core] merging from 23 files and 1 in-memory blocks...

    # Indexar BAM
    samtools index "$SORTED"

    echo "✅ Finalizado: $base"
done

echo "🎉 Proceso completo para todas las muestras."

############## con las muestras faltantes se hizo este ajuste

# Definir rutas
FASTQ_DIR="/home/yordonez3/Danny_OV_WES/concatenated_fastq_faltantes"
REF_GENOME="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"
OUTPUT_DIR="$FASTQ_DIR/BWA_data1"

# Crear carpeta de salida si no existe
mkdir -p "$OUTPUT_DIR"

# Activar entorno conda si lo usas (descomenta y ajusta el nombre si aplica)
# conda activate bwa_env

# Verificar si el índice BWA existe; si no, crearlo
if [[ ! -f "${REF_GENOME}.bwt" ]]; then
    echo "El índice de BWA no se encuentra. Generándolo..."
    bwa index "$REF_GENOME"
else
    echo "Índice de BWA encontrado. Continuando..."
fi

# Alinear todos los pares de archivos FASTQ
for R1 in "$FASTQ_DIR"/*_R1.fastq; do
    SAMPLE=$(basename "$R1" _R1.fastq)
    R2="$FASTQ_DIR/${SAMPLE}_R2.fastq"

    # Verifica que el R2 exista
    if [[ -f "$R2" ]]; then
        echo "Procesando muestra: $SAMPLE"
        bwa mem "$REF_GENOME" "$R1" "$R2" > "$OUTPUT_DIR/${SAMPLE}.sam"
        echo "✓ Alineamiento completado para: $SAMPLE"
    else
        echo "⚠ No se encontró el archivo R2 para la muestra $SAMPLE, se omite."
    fi
done

echo "✅ Todos los alineamientos han terminado."



SyntaxError: invalid syntax. Perhaps you forgot a comma? (3430739851.py, line 9)

In [1]:
## PICARD

#para obtener el archivo de referencia:
## wget http://hgdownload.cse.ucsc.edu/goldenPath/hg38/database/refFlat.txt.gz

input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/BWA_data_faltantes"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/picard_exo"
reference_genome="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

samples=($(ls "${input_dir}"/*.bam | xargs -n 1 basename | sed 's/.bam//'))

# Bucle para procesar cada muestra
for sample_name in "${samples[@]}"; do
    input_bam="${input_dir}/${sample_name}.bam"
    sorted_bam="${output_dir}/${sample_name}_sorted.bam"
    output_file="${output_dir}/${sample_name}_alignment_Metrics.txt"

    # Ordenar el archivo BAM
    java -Xmx64G -jar /home/yordonez3/picard/picard.jar SortSam \
        INPUT="$input_bam" \
        OUTPUT="$sorted_bam" \
        SORT_ORDER=coordinate

    # Recopilar métricas de alineación
    java -Xmx64G -jar /home/yordonez3/picard/picard.jar CollectAlignmentSummaryMetrics \
        REFERENCE_SEQUENCE="$reference_genome" \
        INPUT="$sorted_bam" \
        OUTPUT="$output_file"

    echo "Picard completado para la muestra $sample_name"
done

SyntaxError: invalid syntax (313761294.py, line 10)

In [1]:
# Agregar READ GROUPS a todos los BAM recalibrados

PICARD_JAR="$HOME/picard/picard.jar"  # Cambia esto si tu picard está en otra ruta
bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/picard_exo"
output_dir="${bam_dir}/with_read_groups_faltantes"

mkdir -p "$output_dir"

# ==== PROCESAMIENTO ====
echo "🚀 Agregando Read Groups a los archivos BAM..."

for bam_file in "$bam_dir"/*_dedup.bam; do
  if [[ -f "$bam_file" ]]; then
    filename=$(basename "$bam_file" .bam)
    
    # Datos del read group (puedes adaptarlos si tienes info más precisa)
    RGID="$filename"
    RGLB="lib1"
    RGPL="ILLUMINA"
    RGPU="unit1"
    RGSM="$filename"

    output_bam="${output_dir}/${filename}_rg.bam"

    echo "➡️  Procesando $filename..."

    java -jar "$PICARD_JAR" AddOrReplaceReadGroups \
      I="$bam_file" \
      O="$output_bam" \
      RGID="$RGID" \
      RGLB="$RGLB" \
      RGPL="$RGPL" \
      RGPU="$RGPU" \
      RGSM="$RGSM" \
      SORT_ORDER=coordinate \
      VALIDATION_STRINGENCY=LENIENT \
      CREATE_INDEX=true \
      || { echo "❌ Error al procesar $filename"; continue; }

    echo "✅ Read Group agregado: $output_bam"
  fi
done

echo "🎉 Proceso finalizado. Los BAM con RG están en: $output_dir"

Si faltan archivos por procesar:

PICARD_JAR="$HOME/picard/picard.jar"  # Cambia esto si tu picard está en otra ruta
bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/picard_exo"
output_dir="${bam_dir}/with_read_groups"

mkdir -p "$output_dir"



SyntaxError: invalid syntax. Perhaps you forgot a comma? (1734690619.py, line 7)

In [2]:
# Agregar READ GROUPS a todos los BAM recalibrados

PICARD_JAR="$HOME/picard/picard.jar"  # Cambia esto si tu picard está en otra ruta
bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/picard_exo"
output_dir="${bam_dir}/with_read_groups_faltantes"

mkdir -p "$output_dir"

# ==== PROCESAMIENTO ====
echo "🚀 Agregando Read Groups a los archivos BAM..."

for bam_file in "$bam_dir"/*_dedup.bam; do
  if [[ -f "$bam_file" ]]; then
    filename=$(basename "$bam_file" .bam)
    
    # Datos del read group (puedes adaptarlos si tienes info más precisa)
    RGID="$filename"
    RGLB="lib1"
    RGPL="ILLUMINA"
    RGPU="unit1"
    RGSM="$filename"

    output_bam="${output_dir}/${filename}_rg.bam"

# ❗️ Verifica si el archivo ya existe
    if [[ -f "$output_bam" ]]; then
      echo "⏩ Ya existe: $output_bam (omitido)"
      continue
    fi

    echo "➡️  Procesando $filename..."

    java -jar "$PICARD_JAR" AddOrReplaceReadGroups \
      I="$bam_file" \
      O="$output_bam" \
      RGID="$RGID" \
      RGLB="$RGLB" \
      RGPL="$RGPL" \
      RGPU="$RGPU" \
      RGSM="$RGSM" \
      SORT_ORDER=coordinate \
      VALIDATION_STRINGENCY=LENIENT \
      CREATE_INDEX=true \
      || { echo "❌ Error al procesar $filename"; continue; }

    echo "✅ Read Group agregado: $output_bam"
  fi
done

echo "🎉 Proceso finalizado. Los BAM con RG están en: $output_dir"


SyntaxError: invalid syntax. Perhaps you forgot a comma? (544066919.py, line 7)

In [1]:
## RECALIBRAR BASES

bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/picard_exo/with_read_groups_faltantes"
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"
dbsnp="/home/yordonez3/bioconda3/gatk/Homo_sapiens_assembly38.dbsnp138.vcf"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/rec_bases"
gatk_path="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/BWA_data/apps/gatk-4.6.1.0"

#Importante instalar GATK-versuib ac
conda activate
conda install bioconda::gatk
gatk --version

#Si no funciona, instalar desde la pagina oficial:

https://github.com/broadinstitute/gatk/releases/tag/4.6.1.0
wget https://github.com/broadinstitute/gatk/releases/download/4.6.1.0/gatk-4.6.1.0.zip
unzip gatk-4.6.1.0.zip
gatk-4.6.1.0/gatk
/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/BWA_data/apps/gatk-4.6.1.0

#Instalar java, version reciente:
conda install -c conda-forge openjdk=17
java -version

#si no funciona, instalar desde la pagina principal, descargar la version y en mi terminal:
scp "C:/Users/pmorales/Downloads/OpenJDK11U-jdk_ppc64_aix_hotspot_11.0.26_4.tar.gz" yordonez3@bio-compjord01.biosci.gatech.edu:/home/yordonez3/Danny_OV_WES

## ----------- CREAR DIRECTORIO DE SALIDA SI NO EXISTE -----------
mkdir -p "$output_dir"
# Descargar desde AdoptOpenJDK (ahora Eclipse Temurin) 
wget https://github.com/adoptium/temurin17-binaries/releases/download/jdk-17.0.10%2B7/OpenJDK17U-jdk_x64_linux_hotspot_17.0.10_7.tar.gz -O openjdk17.tar.gz

# Crea el directorio donde se instalará Java:
mkdir -p ~/java

# Descomprimir el archivo descargado en ese directorio:
tar export JAVA_HOME=$HOME/java/jdk-17.0.10+7 

# Configurar las variables de entorno en el archivo .bashrc:
echo export JAVA_HOME=$HOME/java/jdk-17.0.10+7 >> ~/.bashrc
echo 'export PATH=$JAVA_HOME/bin:$PATH' >> ~/.bashrc
export GATK_HOME=$HOME/Danny_OV_WES/concatenated_fastq/BWA_data/apps/gatk-4.6.1.0
export PATH=$GATK_HOME:$PATH

java -version 

# Aplica los cambios en tu sesión actual:
source ~/.bashrc
Verifica que Java esté correctamente instalado:
java -version

# Descargar desde AdoptOpenJDK
wget https://github.com/adoptium/temurin11-binaries/releases/download/jdk-11.0.19%2B7/OpenJDK11U-jdk_x64_linux_hotspot_11.0.19_7.tar.gz

# Iterar sobre los archivos BAM sin duplicados
for bam_file in "$bam_dir"/*-noDups.bam; do

    # Extraer el nombre base del archivo
    base_name=$(basename "$bam_file" -noDups.bam)
    
    # Definir el archivo de salida para los datos de recalibración
    recal_data="${output_dir}/${base_name}-rec_bases"
    
    # Recalibrar bases
    gatk BaseRecalibrator \
        -R "$reference" \
        -I "$bam_file" \
        --known-sites "$dbsnp" \
        -O "$recal_data" \
        --use-original-qualities
        
    # Mensaje de confirmación
    echo "Recalibración de bases completada para $base_name."
done

######## Si no funcionan los pasos anteriores, intentar configurar de la siguiente manera:

# ==== CONFIGURACIÓN ====
JAVA17_BIN="$HOME/java/jdk-17.0.10+7/bin/java"
GATK_JAR="$HOME/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/BWA_data/apps/gatk-4.6.1.0/gatk-package-4.6.1.0-local.jar"

bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/picard_exo/with_read_groups"
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"
dbsnp="/home/yordonez3/gatk-recursos/Homo_sapiens_assembly38.dbsnp138.vcf.gz"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/rec_bases"

# ==== VALIDACIONES ====
if [[ ! -x "$JAVA17_BIN" ]]; then
    echo "❌ No se encontró Java 17 en: $JAVA17_BIN"
    exit 1
fi

if [[ ! -f "$GATK_JAR" ]]; then
    echo "❌ No se encontró el JAR de GATK en: $GATK_JAR"
    exit 1
fi

mkdir -p "$output_dir"

# ==== RECALIBRACIÓN DE BASES ====
echo "🚀 Iniciando recalibración de bases..."

for bam_file in "$bam_dir"/*_rg.bam; do
    if [[ -f "$bam_file" ]]; then
        base_name=$(basename "$bam_file" .bam)
        recal_table="${output_dir}/${base_name}_recal_data.table"

        echo "➡️  Procesando: $base_name"

        "$JAVA17_BIN" -jar "$GATK_JAR" BaseRecalibrator \
            -R "$reference" \
            -I "$bam_file" \
            --known-sites "$dbsnp" \
            -O "$recal_table" \
            --use-original-qualities

        echo "✅ Recalibración completada: $recal_table"
    fi
done

echo "🎉 Proceso de recalibración de bases finalizado."

################################## RECALIBRAR MUESTRAS FALTANTES ############################################
#!/bin/bash

# ======= VARIABLES =======
bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/picard_exo/with_read_groups"
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"
dbsnp="/home/yordonez3/bioconda3/gatk/Homo_sapiens_assembly38.dbsnp138.vcf"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/rec_bases"
gatk_path="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/BWA_data/apps/gatk-4.6.1.0"

mkdir -p "$output_dir"

export PATH="$gatk_path:$PATH"

# ======= PROCESAMIENTO =======
echo "🚀 Iniciando recalibración de bases con GATK..."

for bam_file in "$bam_dir"/*_rg.bam; do
  sample=$(basename "$bam_file" .bam)
  
  # Archivos de salida
  table="$output_dir/${sample}_recal_data.table"
  out_bam="$output_dir/${sample}_recal.bam"

  # Saltar si ya existe
  if [[ -f "$out_bam" ]]; then
    echo "⏩ Ya procesado: $sample (omitido)"
    continue
  fi

  echo "📌 Recalibrando: $sample"

  # Paso 1: BaseRecalibrator
  gatk BaseRecalibrator \
    -I "$bam_file" \
    -R "$reference" \
    --known-sites "$dbsnp" \
    -O "$table" \
    || { echo "❌ Error en BaseRecalibrator: $sample"; continue; }

  # Paso 2: ApplyBQSR
  gatk ApplyBQSR \
    -R "$reference" \
    -I "$bam_file" \
    --bqsr-recal-file "$table" \
    -O "$out_bam" \
    || { echo "❌ Error en ApplyBQSR: $sample"; continue; }

  echo "✅ Recalibración completada: $out_bam"
done

echo "🎉 Todos los BAM han sido recalibrados y están en: $output_dir"

SyntaxError: invalid syntax. Perhaps you forgot a comma? (479067158.py, line 10)

In [2]:
### GENERAR BAM RECALIBRADO

# Rutas base

reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/VARIANTES"
recal_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/VARIANTES"

# Iterar sobre cada archivo .table
for recal_table in "$recal_dir"/*_recal_data.table; do
    # Extraer nombre base correctamente (sin _recal_data.table)
    base_name=$(basename "$recal_table" _recal_data.table)

    bam_file="$bam_dir/${base_name}.bam"
    output_bam="${output_dir}/${base_name}-BQSR.bam"

    # Verificar existencia del BAM original
    if [ ! -f "$bam_file" ]; then
        echo "⚠️ No se encontró el archivo BAM esperado: $bam_file"
        continue
    fi

    # Saltar si el BAM recalibrado ya existe
    if [ -f "$output_bam" ]; then
        echo "⏭️ BAM recalibrado ya existe para $base_name, se salta."
        continue
    fi

    echo "🚀 Recalibrando: $bam_file"
    
    gatk ApplyBQSR \
        -I "$bam_file" \
        -R "$reference" \
        --bqsr-recal-file "$recal_table" \
        -O "$output_bam"

    echo "✅ Recalibración completada para $base_name"
done

echo "🎉 Todos los BAM posibles han sido procesados."

SyntaxError: expected ':' (3211606965.py, line 10)

In [ ]:
#intentar una solución más limpia cuando JAVA no funciona

#Primero, limpia las variables de entorno que ya configuraste:

unset JAVA_HOME
unset JRE_HOME
unset _JAVA_OPTIONS
unset CLASSPATH

#Intenta descargar una versión precompilada de OpenJDK que sea compatible con tu sistema:

mkdir -p ~/java
cd ~/java
wget https://download.java.net/java/GA/jdk17.0.2/dfd4a8d0985749f896bed50d7138ee7f/8/GPL/openjdk-17.0.2_linux-x64_bin.tar.gz
tar -xzf openjdk-17.0.2_linux-x64_bin.tar.gz

#Configura las variables de entorno asegurándote de que no haya conflicto con Miniconda:

echo 'export JAVA_HOME=$HOME/java/jdk-17.0.2' >> ~/.bashrc
echo 'export PATH=$JAVA_HOME/bin:$PATH' >> ~/.bashrc

#Para evitar el problema con las bibliotecas compartidas, añade explícitamente:

echo 'export LD_LIBRARY_PATH=$JAVA_HOME/lib:$JAVA_HOME/lib/server:$LD_LIBRARY_PATH' >> ~/.bashrc

#Aplica los cambios:

source ~/.bashrc

#Verifica la instalación:

java -version
Si el problema persiste, podría ser necesario desactivar temporalmente Miniconda mientras usas Java:
# Antes de usar Java
PATH=$(echo $PATH | sed 's|/home/yordonez3/miniconda3/bin:||')

In [ ]:

##### DESPUES DE ESTE PASO HAY DOS OPCIONES, 
#### SE CONTINUA EL LLAMADO DE VARIANTES SOMATICAS CON MUTEC2 O LLAMADO DE VARIANTES GERMINALES CON HAPLOTYPECALLER (ANCESTRIA GENETICA)



In [ ]:
########################################### VARIANTES SOMATICAS ######################################################
################################################  MUTEC2 ######################################################

## 1 ## LLAMADO DE VARIANTES (cuando solo se tiene tumor)
input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/variantes"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutec2"
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

In [ ]:
#################LLAMADO DE VARIANTES

# 📁 Crear el directorio de salida si no existe
mkdir -p "$output_dir"

# 📄 Obtener nombres base de las muestras
samples=($(ls "$input_dir"/*-BQSR.bam | xargs -n 1 basename | sed 's/-BQSR.bam//'))

# 🔁 Procesar cada muestra individualmente
for sample_name in "${samples[@]}"; do
    input_bam="${input_dir}/${sample_name}-BQSR.bam"
    output_vcf="${output_dir}/${sample_name}.vcf.gz"

    echo "🚀 Ejecutando Mutect2 para la muestra: $sample_name"

    if gatk Mutect2 \
        -R "$reference" \
        -I "$input_bam" \
        -O "$output_vcf" \
        --native-pair-hmm-threads 60 \
        -L chr1 -L chr2 -L chr3 -L chr4 -L chr5 \
        -L chr6 -L chr7 -L chr8 -L chr9 -L chr10 \
        -L chr11 -L chr12 -L chr13 -L chr14 -L chr15 \
        -L chr16 -L chr17 -L chr18 -L chr19 -L chr20 \
        -L chr21 -L chr22 -L chrX -L chrY; then

        echo "✅ Mutect2 completado correctamente para $sample_name"
    else
        echo "❌ ERROR: Falló Mutect2 para $sample_name"
    fi
done

echo "🎯 Todas las muestras han sido procesadas."

## Si se para la corrida:
input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/variantes"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutec2"
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

# Obtener los nombres de las muestras basados en los archivos BAM recalibrados
samples=($(ls "${input_dir}"/*-BQSR.bam | xargs -n 1 basename | sed 's/-BQSR.bam//'))

# Loop para procesar cada muestra con Mutect2 (SI SE TUVO QUE HACER)
for sample_name in "${samples[@]}"; do
    input_bam="${input_dir}/${sample_name}-BQSR.bam"
    output_vcf="${output_dir}/${sample_name}.vcf.gz"
    output_tbi="${output_vcf}.tbi"
    output_stats="${output_vcf}.stats"

    # Comprobar si alguno de los archivos de salida YA EXISTE:
    if [ -f "$output_vcf" ] && [ -f "$output_tbi" ] && [ -f "$output_stats" ]; then
        echo "Los archivos de salida para la muestra $sample_name ya existen. Saltando."
        continue
    fi

    # Ejecutar Mutect2 para llamada de variantes solo tumor, especificando cromosomas
    gatk Mutect2 \
        -R "$reference" \
        -I "$input_bam" \
        -O "$output_vcf" \
        --native-pair-hmm-threads 60 \
        -L chr1 -L chr2 -L chr3 -L chr4 -L chr5 \
        -L chr6 -L chr7 -L chr8 -L chr9 -L chr10 \
        -L chr11 -L chr12 -L chr13 -L chr14 -L chr15 \
        -L chr16 -L chr17 -L chr18 -L chr19 -L chr20 \
        -L chr21 -L chr22 -L chrX -L chrY

    echo "Mutect2 completado para la muestra $sample_name"
done

### Para correr cerrando el pc

input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/variantes"
output_dir="/home/yordonez3/concatenated_fastq/concatenated_all_fastq/mutec2"
reference="/home/yordonez3/Danny_OV_WES/GRCh38_Gencode31/hg38.fa"

# Ejecutar en segundo plano con nohup y redirigir salida a un log
nohup bash -c '
samples=($(ls '"${input_dir}"'/*-BQSR.bam | xargs -n 1 basename | sed "s/-BQSR.bam//"))

for sample_name in "${samples[@]}"; do
    input_bam="'"${input_dir}"'/${sample_name}-BQSR.bam"
    output_vcf="'"${output_dir}"'/${sample_name}.vcf.gz"

    # Ejecutar Mutect2
    gatk Mutect2 \
        -R "'"${reference}"'" \
        -I "$input_bam" \
        -O "$output_vcf" \
        --native-pair-hmm-threads 60 \
        -L chr1 -L chr2 -L chr3 -L chr4 -L chr5 \
        -L chr6 -L chr7 -L chr8 -L chr9 -L chr10 \
        -L chr11 -L chr12 -L chr13 -L chr14 -L chr15 \
        -L chr16 -L chr17 -L chr18 -L chr19 -L chr20 \
        -L chr21 -L chr22 -L chrX -L chrY

    echo "Mutect2 completado para la muestra $sample_name"
done
' > mutect2_llamado_variantes.log 2>&1 &

# Verificar el estado del proceso
ps aux | grep gatk

# Confirmar que el proceso está activo
tail -f mutect2_llamado_variantes.log  # Para salir usa ctrl+c

In [2]:
## 2 ##  FILTRADO DE VARIANTES MUTEC2

### Para conocer los parametros de filtro:
# https://github.com/broadinstitute/gatk/blob/master/docs/mutect/mutect.pdf

input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutec2"
filtered_output_dir="home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutect2_filtered"
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

# Crear el directorio de salida si no existe
mkdir -p "$filtered_output_dir"

# Obtener los nombres de las muestras basados en los archivos VCF sin filtrar de Mutect2
samples=($(ls "${input_dir}"/*.vcf.gz | xargs -n 1 basename | sed 's/.vcf.gz//'))

# Loop para procesar cada muestra con FilterMutectCalls
for sample_name in "${samples[@]}"; do
    unfiltered_vcf="${input_dir}/${sample_name}.vcf"
    filtered_vcf="${filtered_output_dir}/${sample_name}_filtered.vcf"

    # Ejecutar FilterMutectCalls
    gatk FilterMutectCalls \
        -V "$unfiltered_vcf" \
        -R "$reference" \
        -O "$filtered_vcf"

    echo "FilterMutectCalls completado para la muestra $sample_name"
done



## especificar .gz

# Definir rutas
input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutec2"
filtered_output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutect2_filtered"
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

# Crear el directorio de salida si no existe
mkdir -p "$filtered_output_dir"

# Obtener los nombres de las muestras basados en los archivos .vcf.gz
samples=($(ls "${input_dir}"/*.vcf.gz | xargs -n 1 basename | sed 's/.vcf.gz//'))

# Loop para procesar cada muestra con FilterMutectCalls
for sample_name in "${samples[@]}"; do
    unfiltered_vcf="${input_dir}/${sample_name}.vcf.gz"
    filtered_vcf="${filtered_output_dir}/${sample_name}_filtered.vcf"

    # Verificar que el archivo exista antes de procesar
    if [[ -f "$unfiltered_vcf" ]]; then
        gatk FilterMutectCalls \
            -V "$unfiltered_vcf" \
            -R "$reference" \
            -O "$filtered_vcf"

        echo "✅ FilterMutectCalls completado para la muestra $sample_name"
    else
        echo "⚠️  Archivo no encontrado: $unfiltered_vcf"
    fi
done





SyntaxError: invalid syntax (1893628413.py, line 7)

In [ ]:
## NORMALIZACION ANTES DEL MERGE

## NORMALIZACION ANTES DEL MERGE

input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutect2_filtered"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutect2_normalized"

# Crear directorio de salida si no existe
mkdir -p "$output_dir"

# Iterar sobre todos los archivos VCF
for file in "$input_dir"/*.vcf; do
    # Nombre base del archivo
    filename=$(basename "$file")

    # Normalizar el archivo y guardarlo en el directorio de salida
    echo "Normalizando $filename..."
    bcftools norm -m -any -O z -o "$output_dir/${filename}.gz" "$file"

    echo "$filename normalizado."
done


## Para correrlo con el pc cerrado
input_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutect2_filtered"
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/mutect2_normalized"

nohup bash -c '
# Crear directorio de salida si no existe
mkdir -p "'"${output_dir}"'"

# Obtener lista de archivos VCF
samples=($(ls "'"${input_dir}"'"/*.vcf | xargs -n 1 basename))

# Iterar sobre los archivos y normalizarlos
for filename in "${samples[@]}"; do
    input_vcf="'"${input_dir}"'/${filename}"
    output_vcf="'"${output_dir}"'/${filename}"

    echo "Normalizando $filename..."
    
    bcftools norm -m -any -O z -o "$output_vcf" "$input_vcf"

    echo "$filename normalizado."
done
' > normalizacion_vcf.log 2>&1 &

# Verificar el estado del proceso
ps aux | grep bcftools

# Confirmar que el proceso está activo
tail -f normalizacion_vcf.log



##Ver que los archivos mantengan ID ,,
bcftools view -h /home/wovalle3/Samples/total_samples_pool/EXOMA/mutect2_filtered/8T_439645_filtered.vcf.gz | grep "#CHROM"

bcftools query -l /home/wovalle3/Samples/total_samples_pool/EXOMA/mutect2_filtered/8T_439645_filtered.vcf.gz


# Ver encabezado de los archivos normalizados para verificar que quedaron bien
for file in "$output_dir"/*.vcf.gz; do
    bcftools view -h "$file"
done


## Generar el index para poder procesarlos 
for vcf in *.vcf.gz; do
    bcftools index "$vcf"
done



In [ ]:
#### ## ## ## ## ## ## ## ####  HACER MERGE DE LAS MUESTRAS GENERAR UN SOLO ARCHIVO
## REVISAR SCRIPT CARLOS

bcftools merge -m all -O z -o combined_samples.vcf.gz *.vcf.gz


## verificar los enxabezados
bcftools view -h combined_samples.vcf.gz | grep "#CHROM"
## lista de muestras
bcftools query -l combined_samples.vcf.gz
## confirmar que hay variantes en el archivo 
bcftools view combined_samples.vcf.gz | grep -v "^#" | head -n 10


## confirmar el numero de variantes 
bcftools view -H combined_samples.vcf.gz | wc -l
# 75.998.047

bcftools view combined_samples.vcf.gz | grep -v '^#' | wc -l
# 75.998.047

/home/wovalle3/Samples/total_samples_pool/EXOMA/mutect2_filtered/normalized


### Hacer un filtro adicional de calidad
bcftools filter -e "QUAL<30 || INFO/DP<10" combined_samples.vcf.gz -o combined_samples_filtered.vcf.gz

bcftools view combined_samples_filtered.vcf.gz | grep -v '^#' | wc -l
# 8.955.368


### Filtrar el archivo VCF para genes específicos (KRAS, APC, BRAF, TP53)
zgrep -E 'KRAS|APC|BRAF|TP53' combined_samples_filtered.vcf.gz > filtered_genes_variants.vcf.gz

## Extraer las columnas con los genotipos de los pacientes
bcftools view filtered_genes_variants.vcf.gz | cut -f1-9,10- > filtered_genes_variants_with_samples.vcf.gz



##Analizar por paciente
grep '118T_444772' filtered_genes_variants_with_samples.vcf > patient_118T_444772_KRAS_mutations.vcf




In [3]:
### ANOTACION CON ANNOVAR
wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz
tar -zxvf annovar.latest.tar.gz

## descargar bases de datos
perl ./annotate_variation.pl -downdb -buildver hg38 -webfrom annovar refGene /home/wovalle3/annovar/humandb/
perl ./annotate_variation.pl -downdb -buildver hg38 -webfrom annovar refGeneWithVer /home/wovalle3/annovar/humandb/
perl ./annotate_variation.pl -downdb -buildver hg38 -webfrom annovar clinvar_20240917 /home/wovalle3/annovar/humandb/
perl ./annotate_variation.pl -downdb -buildver hg38 -webfrom annovar dbnsfp35a /home/wovalle3/annovar/humandb/
perl ./annotate_variation.pl -downdb -buildver hg38 -webfrom annovar exac03 /home/wovalle3/annovar/humandb/
#perl ./annotate_variation.pl -downdb -buildver hg38 cosmic64 /home/wovalle3/annovar/humandb/



## Anotar variantes
## Primero convertir archivo VCF a formato annovar
perl /home/wovalle3/annovar/convert2annovar.pl -format vcf4 combined_samples_filtered.vcf.gz > annotated_variants2.avinput


## Correr anotaciones 
perl /home/wovalle3/annovar/table_annovar.pl annotated_variants.avinput \
/home/wovalle3/annovar/humandb/ -buildver hg38 \
-out annotated_variants \
-protocol refGeneWithVer,clinvar_20240917,dbnsfp35a,exac03 \
-operation g,f,f,f \
-csvout \
-nopolish \
-remove


## Con este se transforma de una vez y se anota dejando archivos .txt y vfc
perl /home/wovalle3/annovar/table_annovar.pl /home/wovalle3/Samples/total_samples_pool/EXOMA/mutect2_filtered/normalized/combined_samples_filtered.vcf.gz \
/home/wovalle3/annovar/humandb/ -buildver hg38 \
-out annotated_variants \
-protocol refGeneWithVer,clinvar_20240917,dbnsfp35a,exac03 \
-operation g,f,f,f \
-remove \
--vcfinput




multiqc /home/wovalle3/Samples/total_samples_pool/EXOMA/mutect2_filtered/normalized \
/home/wovalle3/Samples/total_samples_pool/EXOMA/snpeff \
/home/wovalle3/Samples/total_samples_pool/EXOMA/annovar \
-o /home/wovalle3/Samples/total_samples_pool/EXOMA/MQCANN

SyntaxError: invalid decimal literal (3183019685.py, line 2)

In [ ]:
## ANOTACIÓN DE VARIANTES CON SNPEFF

# El archivo `annotated_variants.vcf` generado por SnpEff contiene información adicional sobre cada variante. 
# Puedes clasificar las mutaciones según el tipo, impacto funcional, etc.

# Anotación general:
# java -jar /home/wovalle3/snpEff/snpEff.jar ann -c /home/wovalle3/snpEff/snpEff.config -v hg38 6T_439643_filtered.vcf.gz > annotated_variants.vcf

# Anotación de efectos relacionados con cáncer:
java -jar /home/wovalle3/snpEff/snpEff.jar eff -cancer -csvStats stats.csv -v hg38 6T_439643_filtered.vcf.gz > annotated_variants2.vcf

# Bucle para procesar múltiples muestras
input_dir="/home/wovalle3/Samples/total_samples_pool/EXOMA/mutect2_filtered"
output_dir="/home/wovalle3/Samples/total_samples_pool/EXOMA/snpeff"

# Buscar archivos .vcf.gz y procesarlos
for vcf_file in ${input_dir}/*_filtered.vcf.gz; do
    sample_name=$(basename "$vcf_file" .vcf.gz)
    output_vcf="${output_dir}/${sample_name}_annotated.vcf"

    java -jar /home/wovalle3/snpEff/snpEff.jar eff -cancer -csvStats "${output_dir}/${sample_name}_stats.csv" -v hg38 "$vcf_file" > "$output_vcf"

    echo "Anotación completada para la muestra: $sample_name"
done

echo "Anotación de variantes para todas las muestras completada."


#### Si de mutect2 se logra unir el archivo correr entonces:
java -jar /home/wovalle3/snpEff/snpEff.jar eff -cancer -csvStats stats.csv -v hg38 combined_samples.vcf.gz > /home/wovalle3/Samples/total_samples_pool/EXOMA/snpeff/annotated_variants_all.vcf



### ANALISIS DE ARCHIVOS ANOTADOS

# MultiQC para resumen global por muestra
# Filtrar mutaciones de interés como KRAS
grep "KRAS" 8T_439645_filtered_annotated.vcf
grep "KRAS" 8T_439645_filtered_annotated.vcf | wc -l

# Ver todas las variantes con impacto HIGH
grep -i "HIGH" 8T_439645_filtered_annotated.vcf
grep -i "HIGH" 8T_439645_filtered_annotated.vcf | wc -l


In [ ]:
### CONVERTIR VCF A FORMATO TABULAR


vcf_dir="/home/wovalle3/Samples/total_samples_pool/EXOMA/snpeff"
output_dir="/home/wovalle3/Samples/total_samples_pool/EXOMA/snpeff/fil_KRAS"

for vcf_file in "$vcf_dir"/*.vcf; do
    base_name=$(basename "$vcf_file" .vcf)
    gatk VariantsToTable \
        -V "$vcf_file" \
        -F CHROM -F POS -F ID -F REF -F ALT -F QUAL -F FILTER -F AS -F ANN -F GENN \
        -O "$output_dir/$base_name.ann.txt"
done



## Filtrar por mutaciones KRAS
output_dir="/home/wovalle3/Samples/total_samples_pool/EXOMA/snpeff/fil_KRAS"
filtered_dir="/home/wovalle3/Samples/total_samples_pool/EXOMA/snpeff/fil_KRAS"

for file in "$output_dir"/*.ann.txt; do
    base_name=$(basename "$file" .ann.txt)
    grep -i "KRAS" "$file" > "$filtered_dir/$base_name.KRAS_filtered.txt"
done

# Contar las mutaciones de KRAS por muestra
for file in "$filtered_dir"/*KRAS_filtered.txt; do
    echo "$file: $(wc -l < "$file") líneas"
done


In [ ]:
### PARA PODER REALIZAR ANALISIS POSTERIORES CON ANNOVAR O MUTSIG ES NECESARIO COMPRIMIR EN UN SOLO ARCHIVO LOS VCF


## Comprimir los archivos que salen de snpEff

for file in *.vcf; do
    bgzip -c "$file" > "${file}.gz"
done

# Crear un índice
for file in *.vcf.gz; do
    tabix -p vcf "$file"
done

## Fusionar archivos

bcftools merge -o combined.vcf -O v *.vcf.gz




In [ ]:
#### UNA VEZ SE HAGA LA ANOTACION POR ANNOVAR o SNPEFF, SE PROCEDE A USAR LA LIBRERIA EN R MAFTOOLS PARA EL ANALISIS


## Convertir VCF anotado a MAF para poder usar libreria MAFTOOLS

##Previamente preparar objeto VEP
conda update -y -n base -c defaults conda
conda config --set solver libmamba
conda create -y -n vep && conda activate vep
conda install -y -c conda-forge -c bioconda -c defaults ensembl-vep==112.0 htslib==1.20 bcftools==1.20 samtools==1.20 ucsc-liftover==447

mkdir -p $HOME/.vep/homo_sapiens/112_GRCh38/
rsync -avr --progress rsync://ftp.ensembl.org/ensembl/pub/release-112/variation/indexed_vep_cache/homo_sapiens_vep_112_GRCh38.tar.gz $HOME/.vep/
tar -zxf $HOME/.vep/homo_sapiens_vep_112_GRCh38.tar.gz -C $HOME/.vep/
rsync -avr --progress rsync://ftp.ensembl.org/ensembl/pub/release-112/fasta/homo_sapiens/dna_index/ $HOME/.vep/homo_sapiens/112_GRCh38/
vep --version


conda activate vep
## Correr vcf2maf
/home/wovalle3/miniconda3/envs/vep/bin/vcf2maf.pl \
--input-vcf /home/wovalle3/Samples/total_samples_pool/EXOMA/annovar/annotated_variants.hg38_multianno.vcf \
--output-maf /home/wovalle3/Samples/total_samples_pool/EXOMA/annovar/annotated_variants.hg38_multianno.maf \
--ref-fasta /home/wovalle3/Samples/GRCh38_Gencode31/hg38.fa \
--vep-path /home/wovalle3/miniconda3/envs/vep/bin \
--vep-data /home/wovalle3/.vep \
--vep-forks 40


## Probando primero con una sola muestra
/home/wovalle3/miniconda3/envs/vep/bin/vcf2maf.pl \
--input-vcf 8T_439645_filtered_annotated.vcf \
--output-maf 8T_439645_filtered_annotated.maf \
--ref-fasta /home/wovalle3/Samples/GRCh38_Gencode31/hg38.fa \
--vep-path /home/wovalle3/miniconda3/envs/vep/bin \
--vep-data /home/wovalle3/.vep \
--vep-forks 10 \
--species homo_sapiens \
--ncbi-build GRCh38





### CONVERTIR A .MAF EN R
library(maftools)


# Cargar el archivo de variantes anotadas de ANNOVAR
getwd()
setwd("/Users/wen/Library/CloudStorage/OneDrive-InstitutoNacionaldeCancerología/DOCTORADO/TESIS/RNA_seq_todas/Exoma_Ann")
annovar_file <- "annotated_variants.hg38_multianno.csv"
anno_data <- read.csv(annovar_file)

library(dplyr)

# Definir la ruta del archivo anotado por ANNOVAR
annovar_file <- "annotated_variants.hg38_multianno.csv"
anno_data <- read.csv(annovar_file)

# Definir las columnas relevantes para el formato MAF
maf_columns <- c(
  "Chr", "Start", "End", "Ref", "Alt", "Gene.refGeneWithVer", 
  "ExonicFunc.refGeneWithVer", "AAChange.refGeneWithVer", 
  "SIFT_pred", "Polyphen2_HDIV_pred", "CADD_raw", 
  "MutationTaster_pred", "CLNSIG"
)

# Filtrar las columnas relevantes
maf_df <- anno_data %>%
  select(all_of(maf_columns)) %>%
  rename(
    Chromosome = Chr,
    Start_Position = Start, 
    End_Position = End,
    Reference_Allele = Ref, 
    Tumor_Seq_Allele2 = Alt,
    Hugo_Symbol = Gene.refGeneWithVer,
    Variant_Classification = ExonicFunc.refGeneWithVer,
    HGVSp_Short = AAChange.refGeneWithVer,
    SIFT_Prediction = SIFT_pred,
    PolyPhen_Prediction = Polyphen2_HDIV_pred,
    CADD_Score = CADD_raw,
    MutationTaster_Prediction = MutationTaster_pred,
    Clinical_Significance = CLNSIG
  )


# Lista de identificadores de muestras tumorales (ajustar según tus datos)
tumor_ids <- c("10T_439647", "27T_439648", "31T_439649", "3T_439641", "43T_439650", 
               "45T_439651", "48T_439652", "4T_439642", "53T_439654", "57T_439655", 
               "65T_439658", "66T_439659", "67T_439660", "68T_439661", "6T_439643", 
               "72T_439662", "7T_439644", "8T_439645")

# Comprobar la longitud de los identificadores de tumor
length(tumor_ids)

# Asignar los identificadores de tumor al DataFrame
maf_df$Tumor_Sample_Barcode <- rep(tumor_ids, length.out = nrow(maf_df))

# Crear la columna Variant_Type basado en la longitud de Ref y Alt
maf_df <- maf_df %>%
  mutate(Variant_Type = case_when(
    nchar(Reference_Allele) == 1 & nchar(Tumor_Seq_Allele2) == 1 ~ "SNP",
    nchar(Reference_Allele) < nchar(Tumor_Seq_Allele2) ~ "INS",
    nchar(Reference_Allele) > nchar(Tumor_Seq_Allele2) ~ "DEL",
    TRUE ~ NA_character_
  ))

# Función para clasificar variantes según el tipo de mutación
classify_variant <- function(x) {
  if (x == "synonymous SNV") {
    return(NA)
  } else if (x == "nonsynonymous SNV") {
    return("Missense_Mutation")
  } else if (x == "nonframeshift deletion") {
    return("In_Frame_Del")
  } else if (x == "frameshift deletion") {
    return("Frame_Shift_Del")
  } else if (x == "nonframeshift insertion") {
    return("In_Frame_Ins")
  } else if (x == "frameshift insertion") {
    return("Frame_Shift_Ins")
  } else if (x == "stopgain" || x == "stoploss") {
    return("Nonsense_Mutation")
  } else if (x == "startloss") {
    return("Nonsense_Mutation")
  } else if (x == "unknown") {
    return(NA)
  } else {
    return("Splice_Site")
  }
}

# Aplicar la clasificación de variantes
maf_df$Variant_Classification <- sapply(maf_df$Variant_Classification, classify_variant)

# Filtrar las filas donde la clasificación de variante no es NA
maf_df <- maf_df[!is.na(maf_df$Variant_Classification), ]

# Reordenar las columnas según el formato MAF
maf_columns_order <- c(
  "Hugo_Symbol", "Chromosome", "Start_Position", "End_Position",
  "Reference_Allele", "Tumor_Seq_Allele2", "Variant_Classification",
  "HGVSp_Short", "Variant_Type", "Tumor_Sample_Barcode",
  "SIFT_Prediction", "PolyPhen_Prediction", "CADD_Score",
  "MutationTaster_Prediction", "Clinical_Significance"
)

# Asegurarse de que las columnas que existen en maf_df estén en el orden adecuado
maf_df <- maf_df %>%
  select(all_of(maf_columns_order))
head(maf_df)

# Guardar el archivo en formato MAF (tab-separated)
output_file <- "output_maf_file.maf"
write.table(maf_df, file = output_file, sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)



install.packages("BiocManager")

BiocManager::install("maftools")
library(maftools)


## Leer el .maf
getwd()
setwd("/home/wovalle3/Samples/total_samples_pool/EXOMA/mutect2_filtered/normalized")
maf_data <- read.maf(maf = "output_maf_file.maf", verbose = FALSE)


## Analisis
maf_summary(maf_data)
## Primeras filas 
head(getSampleSummary(maf_data))
getSampleSummary(maf_data)

## Gene summary
getGeneSummary(maf_data)

## como coldata
getFields(maf_data)



## Graficos 
##  Plotting MAF summary.
plotmafSummary(maf = maf_data, rmOutlier = TRUE, addStat = 'median', dashboard = TRUE, titvRaw = FALSE)

mafbarplot(maf = maf_data, top = 10)

## Oncoplots
oncoplot(maf = maf_data, top = 10)


laml.titv = titv(maf = maf_data, plot = FALSE, useSyn = TRUE)
#plot titv summary
plotTiTv(res = laml.titv)


## Lollipop plots for amino acid changes
#lollipop plot for DNMT3A, which is one of the most frequent mutated gene in Leukemia.
lollipopPlot(
  maf = maf_data,
  gene = 'KRAS',
  AACol = 'Protein_Change',
  showMutationRate = TRUE,
  labelPos = 882
)


### Plotting VAF (Variant Allele Frequencies)
plotVaf(maf = maf_data, vafCol = 'i_TumorVAF_WU')


# Generar el gráfico y guardarlo en un archivo PNG
png("/home/wovalle3/Samples/total_samples_pool/EXOMA/plots/maf_summary.png", width = 800, height = 600)
plotmafSummary(maf_data)
dev.off()  # Finaliza la escritura en el archivo PNG




### para convertir archivo annovar a maf
getwd()
setwd("/home/wovalle3/Samples/total_samples_pool/EXOMA/mutect2_filtered/normalized")

annovar_file <- "annotated_variants.hg38_multianno.csv"
annovar_file2 <- "annotated_variants.refGene.variant_function"


maf_file <- annovarToMaf(annovar = annovar_file2,
                         refBuild = "hg38",        # Especifica el genoma de referencia (puede ser "hg38" o "hg19")
                         table = "refGene",        # Tabla de referencia (puede ser "refGene" o "ensGene")
                         #header = TRUE,            # Si el archivo ANNOVAR tiene encabezado, ponlo en TRUE
                         sep = "\t",               # Separador de campos (tabulado por defecto)
                         MAFobj = FALSE,           # Si FALSE, devuelve el archivo MAF como un data frame
                         basename = "output")

In [1]:

################################################ VARIANTES GERMINALES ######################################################
################################################  HAPLOTYPECALLER ######################################################

## LLAMADO DE VARIANTES GERMINALES

mkdir -p "gvcfs"

# Ruta al archivo de referencia
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

# Directorio donde están los BAM con BQSR
bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/VARIANTES"

# Directorio donde se guardarán los GVCF generados
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/VARIANTES/gvcfs"

# Archivo dbSNP
dbsnp_file="/home/yordonez3/gatk-recursos/Homo_sapiens_assembly38.dbsnp138.vcf"

# Ruta al archivo GATK local
gatk_jar="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/gatk-4.6.1.0/gatk-package-4.6.1.0-local.jar"

# Número de hilos a usar
num_threads=14

# Iterar sobre los BAM ya recalibrados
for bam_file in "$bam_dir"/*-BQSR.bam; do
  sample_name=$(basename "$bam_file" -BQSR.bam)
  output_gvcf="$output_dir/gvcfs/${sample_name}.g.vcf.gz"

  # Verificar si ya existe el archivo .g.vcf.gz
  if [[ -f "$output_gvcf" ]]; then
    echo "Archivo GVCF ya existe para $sample_name, se omite."
    continue
  fi

  java -Xmx35g -jar "$gatk_jar" HaplotypeCaller \
    -R "$reference" \
    -I "$bam_file" \
    -O "$output_gvcf" \
    -ERC GVCF \
    --dbsnp "$dbsnp_file" \
    --native-pair-hmm-threads "$num_threads"

  echo "HaplotypeCaller completado para $sample_name."
done

## Cerrando el pc

# Ruta al archivo de referencia
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

# Directorio donde están los BAM con BQSR
bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/BAM_rec"

# Directorio donde se guardarán los GVCF generados
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/variantes_germinales"

# Archivo dbSNP
dbsnp_file="/home/yordonez3/gatk-recursos/Homo_sapiens_assembly38.dbsnp138.vcf"

# Ruta al archivo GATK local
gatk_jar="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/gatk-4.6.1.0/gatk-package-4.6.1.0-local.jar"

num_threads=14

mkdir -p "$output_dir/gvcfs"

nohup bash -c "
for bam_file in $output_dir/*-BQSR.bam; do
  sample_name=\$(basename \$bam_file -BQSR.bam)

  java -Xmx35g -jar "$gatk_jar" HaplotypeCaller \
    -R \"$reference\" \
    -I \"\$bam_file\" \
    -O \"$output_dir/gvcfs/\${sample_name}.g.vcf.gz\" \
    -ERC GVCF \
    --dbsnp \"$dbsnp_file\" \
    --native-pair-hmm-threads \"$num_threads\"

  echo \"HaplotypeCaller completado para \$sample_name.\"
done
" > "$output_dir/llamado_de_variantes.log" 2>&1 &

#### Para verificar que los archivos vcf se generaron bien:

##### Revisa todos los archivos .g.vcf.gz en el directorio actual
for file in *.g.vcf.gz; do
    echo "🔍 Verificando $file..."

    # Verifica si el archivo está bien comprimido con tabix
    if tabix -p vcf "$file" 2>&1 | grep -q "failed"; then
        echo "❌ Archivo corrupto o mal comprimido: $file"
    else
        echo "✅ Archivo válido: $file"
    fi
done

MODIFICADO:

# Ruta al archivo de referencia
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

# Directorio donde están los BAM con BQSR
bam_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/variantes"

# Directorio donde se guardarán los GVCF generados
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/variantes_germinales"

# Archivo dbSNP
dbsnp_file="/home/yordonez3/gatk-recursos/Homo_sapiens_assembly38.dbsnp138.vcf"

# Ruta al archivo GATK local
gatk_jar="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/gatk-4.6.1.0/gatk-package-4.6.1.0-local.jar"

# Ejecutar todo el proceso con nohup para que siga corriendo en segundo plano
nohup bash -c '

# Iterar sobre los BAM ya recalibrados
for bam_file in "$bam_dir"/*-BQSR.bam; do
  sample_name=$(basename "$bam_file" -BQSR.bam)
  output_vcf="$output_dir/gvcfs/${sample_name}.g.vcf.gz"

  # Verificar si el archivo .g.vcf.gz ya existe
  if [ -f "$output_vcf" ]; then
    echo "El archivo $output_vcf ya existe, se omite el HaplotypeCaller para $sample_name." >> nohup.out
    continue  # Salta al siguiente BAM si ya existe el archivo de salida
  fi

  # Ejecutar HaplotypeCaller si no existe el archivo de salida
  java -Xmx35g -jar "$gatk_jar" HaplotypeCaller \
    -R "$reference" \
    -I "$bam_file" \
    -O "$output_vcf" \
    -ERC GVCF \
    --dbsnp "$dbsnp_file" \
    --native-pair-hmm-threads "$num_threads"

  echo "HaplotypeCaller completado para $sample_name." >> nohup.out
done
' &

SyntaxError: unterminated string literal (detected at line 68) (3964709590.py, line 68)

In [ ]:
## GENOMIC IMPORT

# Ruta al genoma de referencia
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

# Directorio de salida
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/PANEL/gvcfs_panel"

# Ejecutable GATK (usando versión 4.2.2.0)
gatk="$HOME/miniconda3/bin/java -jar /home/yordonez3/gatk-4.2.2.0/gatk-package-4.2.2.0-local.jar"

# Ruta a los archivos gVCF
gvcf_dir="$output_dir/gvcfs_panel"

# Crear el directorio temporal si no existe
tmp_dir="$gvcf_dir/tmp"
mkdir -p "$tmp_dir"

# Verificar permisos de lectura y escritura
if [ ! -r "$tmp_dir" ] || [ ! -w "$tmp_dir" ]; then
  echo "❌ Error: No tienes permisos de lectura/escritura en $tmp_dir"
  exit 1
fi

# Verificar si hay archivos g.vcf.gz
gvcf_files=("$gvcf_dir"/*.g.vcf.gz)
if [ ! -e "${gvcf_files[0]}" ]; then
  echo "⚠️ No se encontraron archivos .g.vcf.gz en $gvcf_dir"
  exit 1
fi

# Mostrar cuántas muestras encontró
echo "🔎 Se encontraron ${#gvcf_files[@]} muestras para importar."

# Construir lista de inputs
inputs=""
for file in "${gvcf_files[@]}"; do
  inputs+=" -V $file"
done

# Lista de cromosomas
chromosomes=(chr{1..22} chrX chrY)

# Ejecutar por cada cromosoma solo si no se ha procesado
for chr in "${chromosomes[@]}"; do
  workspace_path="$output_dir/my_database_$chr"
  if [ -d "$workspace_path" ]; then
    echo "✅ $chr ya fue procesado. Omitiendo..."
  else
    echo "🚀 Procesando $chr ..."
    $gatk GenomicsDBImport \
      $inputs \
      --genomicsdb-workspace-path "$workspace_path" \
      --tmp-dir "$tmp_dir" \
      --intervals "$chr"

    # Verifica si se creó correctamente
    if [ $? -eq 0 ]; then
      echo "✅ $chr importado correctamente."
    else
      echo "❌ Error al importar $chr."
    fi
  fi
done

echo "🎉 Proceso completo."


In [1]:
####GENOMIC IMPORT PARA REVISAR ARVHIVOS
#!/bin/bash

# Ruta al genoma de referencia
reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

# Directorio de salida
output_dir="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/variantes_germinales"

# Ejecutable GATK (versión 4.2.2.0)
gatk="$HOME/miniconda3/bin/java -jar /home/yordonez3/gatk-4.2.2.0/gatk-package-4.2.2.0-local.jar"

# Ruta a los archivos gVCF
gvcf_dir="$output_dir/gvcfs"

# Crear el directorio temporal si no existe
tmp_dir="$gvcf_dir/tmp"
mkdir -p "$tmp_dir"

# Verificar permisos de lectura y escritura
if [ ! -r "$tmp_dir" ] || [ ! -w "$tmp_dir" ]; then
  echo "❌ Error: No tienes permisos de lectura/escritura en $tmp_dir"
  exit 1
fi

# Verificar si hay archivos g.vcf.gz
gvcf_files=("$gvcf_dir"/*.g.vcf.gz)
if [ ! -e "${gvcf_files[0]}" ]; then
  echo "⚠️ No se encontraron archivos .g.vcf.gz en $gvcf_dir"
  exit 1
fi

# Mostrar cuántas muestras encontró
echo "🔎 Se encontraron ${#gvcf_files[@]} muestras para importar."

# Construir lista de inputs
inputs=""
for file in "${gvcf_files[@]}"; do
  inputs+=" -V $file"
done

# Lista completa de cromosomas
chromosomes=(chr{1..22} chrX chrY)

for chr in "${chromosomes[@]}"; do
  workspace_path="$output_dir/my_database_$chr"
  
  # Eliminar workspace corrupto si existe
  if [ -d "$workspace_path" ]; then
    echo "🧹 Borrando workspace corrupto existente para $chr ..."
    rm -rf "$workspace_path"
  fi

  echo "🚀 Importando $chr ..."
  
  # Ejecutar GenomicsDBImport con log específico
  $gatk GenomicsDBImport \
    $inputs \
    --genomicsdb-workspace-path "$workspace_path" \
    --tmp-dir "$tmp_dir" \
    --intervals "$chr" \
    &> "$output_dir/log_GenomicsDBImport_$chr.txt"

  # Verificar resultado
  if [ $? -eq 0 ]; then
    echo "✅ $chr importado correctamente."
  else
    echo "❌ Error al importar $chr. Revisa $output_dir/log_GenomicsDBImport_$chr.txt"
  fi
done

echo "🎉 Proceso completo."

In [2]:
## GENOTIPIFICACION

reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

for chr in {1..22} X Y; do
  echo "🧬 Procesando chr$chr..."
  gatk GenotypeGVCFs \
    -R "$reference" \
    -V gendb://my_database_chr$chr \
    -O genotipos_chr${chr}.vcf.gz
done


SyntaxError: invalid syntax. Perhaps you forgot a comma? (984889335.py, line 5)

In [ ]:
## 10. FILTRADO DE VARIANTES

reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

for chr in {1..22} X Y; do
  input_vcf="genotipos_chr${chr}.vcf.gz"
  output_vcf="genotipos_chr${chr}_filtered.vcf.gz"
  
  if [ ! -s "$input_vcf" ]; then
    echo "⚠️ Archivo $input_vcf no existe o está vacío, se omite."
    continue
  fi

  echo "🧬 Filtrando variantes de $input_vcf..."

  gatk VariantFiltration \
    -R "$reference" \
    -V "$input_vcf" \
    --window 35 \
    --cluster 3 \
    --filter-name "FS" --filter "FS > 30.0" \
    --filter-name "QD" --filter "QD < 2.0" \
    --filter-name "MQ" --filter "MQ < 30.0" \
    -O "$output_vcf"
done

# Contar los SNPs obtenidos
bcftools view -H -v snps geno_exo_28_filtered.vcf.gz | wc -l
for chr in {1..22} X Y; do
  echo "Cromosoma $chr:"
  bcftools view -H -v snps genotipos_chr${chr}_filtered.vcf.gz | wc -l
done


#Criterios para el filtrado
FS (Phred-scaled p-value using Fisher's exact test to detect strand bias)
FS > 30.0: Filters variants with a Fisher Strand (FS) score greater than 30.0.

QD es el valor de calidad de una variante dividido por la profundidad de cobertura en ese sitio específico
QD < 2.0: Filters variants with a Quality by Depth (QD) score less than 2.0.
variantes con un valor QD menor que 2.0 serán marcadas como filtradas.
    
MQ Mapping Quality MQ ≥ 30, es un umbral comúnmente utilizado en muchos estudios. 
Se considera que un MQ de 30 o más indica una alineación confiable.

In [ ]:
###PLINK

mv geno_exo_28_filtered.vcf.gz geno_exo_28_filtered.vcf.gz.tbi /home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink

#Antes de empezar revisar que el archivo tenga los IDs de los SNPs
bcftools view --no-header geno_exo_28_filtered.vcf.gz | head

#En dado caso, anotar con bcftools
mv Homo_sapiens_assembly38.dbsnp138.vcf.gz /home/wovalle3/LREY/variant_call/Plink_Rnaseq
bcftools index Homo_sapiens_assembly38.dbsnp138.vcf.gz

bcftools annotate --annotations Homo_sapiens_assembly38.dbsnp138.vcf.gz --columns ID --output geno_final.vcf.gz --output-type z geno_filt_annot.vcf.gz
bcftools view --no-header geno_final.vcf.gz | head

#La anotacion no permite el merge entre las poblaciones de referencia. Hacer sin anotar

In [ ]:
###PLINK
VCF de entrada: geno_exo_28_filtered.vcf.gz

bcftools view --no-header geno_exo_28_filtered.vcf.gz | head

##1. Generar los bfiles con el archivo .vcf de mis muestras
The first step for quality control (QC) analysis in PLINK starting from
your VCF files is to convert the VCF file to PLINK format (BED, BIM, FAM).
PLINK provides a tool called plink2 for this purpose. 
                                 
plink --vcf geno_exo_28_filtered.vcf.gz --make-bed --out geno_plink

#Results
Total genotyping rate is 0.461463.
14.853.128 variants and 18 people pass filters and QC.

Total genotyping rate is 0.434183.
20.581.021 variants and 28 people pass filters and QC.

In [ ]:
#2. Perform QC analysis

#Opciones de QC
#Removing individuals with high missing genotype rates (--mind): --mind 0.05
#Removing SNPs with high missingness (--geno): --geno 0.05
#Removing SNPs with low minor allele frequency (--maf): --maf 0.01
#Performing Hardy-Weinberg equilibrium (HWE) testing (--hwe): 1e-6
#Removing SNPs with extreme heterozygosity (--het).
#Identificación de SNPs Duplicados (--list-duplicate-vars)
#Filtrar por Identidad por Descendencia (IBD) (--genome y --rel-cutoff)

#Usar comando geno 0.1 para que deje solamente SNPs con CR>90%
plink --bfile geno_plink --geno 0.1  --make-bed --out geno_plink_fil1

#Results
Total genotyping rate is 0.461463.
12.763.929 variants removed due to missing genotype data (--geno).
2.089.199 variants and 18 people pass filters and QC.

Total genotyping rate is 0.434183.
18.721.953 variants removed due to missing genotype data (--geno).
1.859.068 variants and 28 people pass filters and QC.

#Control de calidad para individuos con mind. Deja solamente casos CR>90%
plink --bfile geno_plink_fil1 --mind 0.25 --make-bed --out geno_plink_fil2   ##Mejor hacerlo con 0.1 despues

#Results
Total genotyping rate in remaining samples is 0.943978.
2.089.199 variants and 17 people pass filters and QC.

Total genotyping rate is 0.97497.
1.859.068 variants and 28 people pass filters and QC.

In [ ]:
#3. Filtrar por MAF y HW

#MAF: 0.01 remueve variantes con frecuencia de alelo menor a 1% (es dcir mutaciones y demas)
plink --bfile geno_plink_fil2 --maf 0.01 --make-bed --out geno_plink_fil3

#Results
Total genotyping rate is 0.943978
124.280 variants removed due to minor allele threshold(s)
1.964.919 variants and 17 people pass filters and QC.

Total genotyping rate is 0.97497.
3041 variants removed due to minor allele threshold(s)
1.856.027 variants and 28 people pass filters and QC


#HW
plink --bfile geno_plink_fil3 --hwe 1e-6 --hwe-all --make-bed --out geno_plink_fil4

#Results
Total genotyping rate is 0.944006.
--hwe: 960 variants removed due to Hardy-Weinberg exact test.
1.963.959 variants and 17 people pass filters and QC.

Total genotyping rate is 0.974968.
--hwe: 520 variants removed due to Hardy-Weinberg exact test.
1.855.507 variants and 28 people pass filters and QC.

In [ ]:
# Perform LD pruning in two steps:
# Step 1: Generate list of SNPs to keep
# We perform linkage disequilibrium (LD) pruning using:
  # - a window size of 50 SNPs
  # - a step size of 5 SNPs
  # - r2 threshold of 0.2 (correlation between SNPs)

plink --bfile geno_plink_fil4 --indep-pairwise 50 5 0.2 --out ld_pruned

#Results 
Total genotyping rate is 0.944005.
1.963.959 variants and 17 people pass filters and QC.
Pruning complete.  1.668.064 of 1.963.959 variants removed.


# Step 2: Apply LD pruning
plink --bfile geno_plink_fil4 --extract ld_pruned.prune.in --make-bed --out geno_plink_fil5

#Results 
Ambiguous sex IDs written to geno_plink_fil5.nosex .
--extract: 1963959 variants remaining.
Warning: At least 295894 duplicate IDs in --extract file.
Total genotyping rate is 0.944005.
1.963.959 variants and 17 people pass filters and QC.

Total genotyping rate is 0.974963.
1.855.507 variants and 28 people pass filters and QC


In [ ]:
#### CODIGO ALEJANDRO

!./plink2 --bfile exome_data \
--maf 0.01 \
--geno 0.1 \
--mind 0.1 \
--hwe 0.0001 \
--make-bed \
--out admixture_maf-0.01_geno-0.1_hwe-1e4

# Perform LD pruning in two steps:
# Step 1: Generate list of SNPs to keep
# We perform linkage disequilibrium (LD) pruning using:
  # - a window size of 50 SNPs
  # - a step size of 5 SNPs
  # - r2 threshold of 0.2 (correlation between SNPs)
!./plink2 --bfile admixture_maf-0.01_geno-0.1_hwe-1e4 \
       --indep-pairwise 50 5 0.2 \
       --out ld_pruned

# Step 2: Apply LD pruning
!./plink2 --bfile admixture_maf-0.01_geno-0.1_hwe-1e4 \
       --extract ld_pruned.prune.in \
       --make-bed \
       --out admixture_LD_QC

In [ ]:
#4. Set-missing-var-ids para mis datos y el panel de referencia

#El comando set-missing-var-ids en PLINK se utiliza para asignar 
#identificadores (IDs) a los SNPs que no tienen un ID en el archivo BIM

plink --bfile geno_plink_fil5 --set-missing-var-ids chr@:#:\$2:\$1 --make-bed --out genoexo_NoMissVar
    
#Results
Total genotyping rate is 0.944005.
1.963.959 variants and 17 people pass filters and QC.

Total genotyping rate is 0.974963.
1.855.507 variants and 28 people pass filters and QC.

head genoexo_NoMissVar.bim
#results 
1	chr1:14671:G:A	0	14671	A	G
1	chr1:14677:G:A	0	14677	A	G
1	chr1:14685:T:C	0	14685	T	C
1	chr1:14758:G:A	0	14758	A	G    


#Esto lo hago desde la carpeta de FinalReferencePanel si no se tiene 
#plink --bfile extractedChrAll --set-missing-var-ids chr@:#:\$2:\$1 --make-bed --out extractedChrAll_NoMissVar
    
#Results
23.886.383 variants loaded from .bim file.
969 people (0 males, 0 females, 969 ambiguous) loaded from .fam.
Total genotyping rate is 0.999872.
23.886.383 variants and 969 people pass filters and QC.

#Mover los archivos a la carpeta desde donde se esta haciendo el analisis
mv extractedChrAll_NoMissVar* /home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink

In [ ]:
#5. Encontrar variantes que se comparten entre la base de referencia y la mia

#Para asegurarte de que el panel de referencia tiene exactamente las mismas 
#variantes que tu archivo antes de realizar un merge en PLINKls

comm -12 <(sort genoexo_NoMissVar.bim) <(sort extractedChrAll_NoMissVar.bim) > variantes_comunes.txt
wc -l variantes_comunes.txt #119.776 compartidas  #111.293

plink --bfile extractedChrAll_NoMissVar --extract variantes_comunes.txt --make-bed --out Ref_comunes

#Results
Total genotyping rate is 0.99987.
119.776 variants and 969 people pass filters and QC

Total genotyping rate is 0.999874.
111.293 variants and 969 people pass filters and QC.

plink --bfile genoexo_NoMissVar --extract variantes_comunes.txt --make-bed --out genoexo_comunes
#Results
Total genotyping rate is 0.95625.
119.776 variants and 17 people pass filters and QC.

Total genotyping rate is 0.978565.
111.293 variants and 28 people pass filters and QC.

In [ ]:
#6. Merge del panel de referencia con mis datos

plink --bfile Ref_comunes --bmerge genoexo_comunes --make-bed --out merged_set_28

(969 + 17)
Total genotyping rate is 0.999118.
119.776 variants and 986 people pass filters and QC.

Total genotyping rate is 0.999275.
111.293 variants and 997 people pass filters and QC.

plink --bfile merged_set_28 --geno 0.10  --make-bed --out merged_set_fil28

#results
0 variants removed due to missing genotype data (--geno).
119776 variants and 986 people pass filters and QC.

0 variants removed due to missing genotype data (--geno).
111293 variants and 997 people pass filters and QC

#PCA con los resultados
plink --bfile merged_set_fil28 --pca --out pca_results_28
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/pca_results_28.eigenval /Users/wen/Documents/
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/pca_results_28.eigenvec /Users/wen/Documents/

#Los archivos generados por PLINK para el análisis de PCA (pca_results.eigenvec y pca_results.eigenval) 
#son archivos de texto que contienen los resultados del análisis de componentes principales. 
#Puedes abrir y analizar estos archivos en R para visualización y análisis adicionales

In [ ]:
#9.	Generar un nuevo archivo recodificado para Admixture

plink --bfile merged_set_fil28 --recode12 --out Final_admixture28

#results
Total genotyping rate is 0.999118.
119.776 variants and 986 people pass filters and QC.

Total genotyping rate is 0.999275.
111.293 variants and 997 people pass filters and QC.

#Instalar Admixture en el servidor 
wget https://dalexander.github.io/admixture/binaries/admixture_linux-1.3.0.tar.gz

#Descomprimir el archivo tar.gz
tar -zxvf admixture_linux-1.3.0.tar.gz

In [ ]:
#10. Correr admixture desde donde esta el archivo ped
/home/wovalle3/ADMIXTURE/dist/admixture_linux-1.3.0/admixture Final_admixture28.ped 3

#Results
Summary: 
Converged in 13 iterations (239.995 sec)
Loglikelihood: -85308375.284505
Fst divergences between estimated populations: 
	Pop0	Pop1	
Pop0	
Pop1	0.136	
Pop2	0.181	0.123	
Writing output files.

Converged in 16 iterations (249.68 sec)
Loglikelihood: -71583396.008181
Fst divergences between estimated populations: 
	Pop0	Pop1	
Pop0	
Pop1	0.134	
Pop2	0.182	0.126	
Writing output files.

#Pasar archivo Final_admixture.3.Q y merged_set.fam a pc para mirar resultados
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/Final_admixture28.3.Q /Users/wen/Documents/
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/merged_set_28.fam /Users/wen/Documents/